In [20]:
globals().clear()

In [21]:
import sys
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit

In [22]:
df = pd.read_csv("/Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Code/data/coffee_data.csv")

### show data for the first 5 lines

In [23]:
for fname in [
    "/Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Code/data/coffee_data.csv" # for fname in [...] 表示循环读取列表里的每个元素，并把它赋值给 fname
]:
    print(f"\n>>> {fname} First 3 Lines:")  # f-string：f"..." 是 Python 的格式化字符串，可以在字符串里直接插入变量。  \n 表示换行
    print(pd.read_csv(fname, nrows=5))     # read_csv() 是 pandas 最常用的函数之一，用来读取 CSV 文件并返回一个 DataFrame（类似表格的数据结构）。


>>> /Users/terrylu/Desktop/UF/Courses/2025-2026/IO/IO_Code/data/coffee_data.csv First 3 Lines:
   price_starbucks  price_dunkin  time_starbucks  time_dunkin     income  \
0         5.195254      4.993072              10           20  60.313563   
1         5.860757      2.720811               5           19  84.390030   
2         5.411054      3.556093               8           14  65.918877   
3         5.179533      2.150401               5            6  54.251162   
4         4.694619      2.047151               5           11  71.886693   

   choice_is_starbucks  
0                    1  
1                    0  
2                    1  
3                    0  
4                    0  


In [24]:
print(df.describe().T[['mean', 'std', 'min', 'max']])   # df.describe() 作用：返回 DataFrame 的统计摘要（summary statistics）。
# .T 是转置操作，把行和列互换。[['mean', 'std', 'min', 'max']] 选择特定的列进行显示。

                          mean        std       min         max
price_starbucks       4.985836   1.158422  3.000290    6.999912
price_dunkin          3.980958   1.165219  2.000666    5.999848
time_starbucks       12.511100   4.594604  5.000000   20.000000
time_dunkin          12.434200   4.590226  5.000000   20.000000
income               59.828028  15.027648 -9.899295  114.084881
choice_is_starbucks   0.324900   0.468361  0.000000    1.000000


Income is in 1000s

Travel time is in minutes

price is in dollars

Individual level data

Income varianle will be interacted with price

In [25]:
summary_by_choice = df.groupby('choice_is_starbucks').agg({         # df.groupby('choice_is_starbucks') 作用：按照 choice_is_starbucks 这一列的值来分组。
                                                                    # .agg({...}) 作用：对每组数据计算不同列的统计指标。 含义：对每组，分别计算这些列的平均值。
    'price_starbucks': 'mean',                                     # 例如：price_starbucks → 每组 Starbucks 的平均价格。 time_dunkin → 每组 Dunkin 的平均时间。 income → 每组消费者的平均收入
    'price_dunkin': 'mean',
    'time_starbucks': 'mean',
    'time_dunkin': 'mean',
    'income': 'mean'
    }).rename(index={0: 'Chose Dunkin', 1: 'Chose Starbucks'})    # rename(index={0: 'Chose Dunkin', 1: 'Chose Starbucks'}) 作用：把索引 0 和 1 分别重命名为 'Chose Dunkin' 和 'Chose Starbucks'。

print(summary_by_choice)

                     price_starbucks  price_dunkin  time_starbucks  \
choice_is_starbucks                                                  
Chose Dunkin                5.331770      3.634030       12.976152   
Chose Starbucks             4.267028      4.701831       11.544783   

                     time_dunkin     income  
choice_is_starbucks                          
Chose Dunkin           11.988891  59.666324  
Chose Starbucks        13.359495  60.164027  


create new variable for differences between product characteristics

In [26]:
df['price_diff'] = df['price_starbucks'] - df['price_dunkin']         # 新建一列 price_diff，表示 Starbucks 和 Dunkin 价格的差值
df['time_diff'] = df['time_starbucks'] - df['time_dunkin']        # 新建一列 time_diff，表示 Starbucks 和 Dunkin 所需时间的差值
df['price_income_interaction'] = df['price_starbucks'] * df['income']- df['price_dunkin'] * df['income']  # 新建一列 price_income_interaction，表示价格与收入的交互项： Starbucks 支出（价格 × 收入） − Dunkin 支出（价格 × 收入）

y = df['choice_is_starbucks']               # 因变量 y，表示消费者是否选择 Starbucks（1 表示选择 Starbucks，0 表示选择 Dunkin）
X = df[['price_diff', 'time_diff', 'price_income_interaction']]    # 自变量 X，包含价格差异、时间差异和价格与收入的交互项

note: no inercept here

In [27]:
logit_model = Logit(y, X)                       # 创建逻辑回归模型对象

logit_result = logit_model.fit(disp=True)       # 拟合模型，disp=True 显示迭代过程

# logit_result 不是普通的数组，而是一个 结果对象 (results object)。
# 这个对象里保存了模型的所有结果，包括：
# 系数估计：logit_result.params
# 标准误差：logit_result.bse
# 对数似然：logit_result.llf
# 预测方法：logit_result.predict(X)
# 显著性检验结果：logit_result.pvalues
# 完整报告：logit_result.summary()

print(logit_result.summary())                  # 输出模型摘要

Optimization terminated successfully.
         Current function value: 0.363798
         Iterations 7
                            Logit Regression Results                           
Dep. Variable:     choice_is_starbucks   No. Observations:                10000
Model:                           Logit   Df Residuals:                     9997
Method:                            MLE   Df Model:                            2
Date:                 Sun, 21 Sep 2025   Pseudo R-squ.:                  0.4230
Time:                         01:51:36   Log-Likelihood:                -3638.0
converged:                        True   LL-Null:                       -6305.1
Covariance Type:             nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
price_diff                  -1.5569      0.109    -14.332      0.000    

price_income_interaction     0.0024: 

higer income people are less price sensitive

why: they care less about price

math why: the coefficient is positive, so as income increases, the negative effect of price_diff on the utility of choosing Starbucks is mitigated.

### With Robust Covariance Matrix
现在在用 稳健标准误 (robust standard errors) 来改进 Logit 模型的结果

In [28]:
logit_result_robust = logit_model.fit(disp=True, cov_type='HC0')  # HC0 for robust standard errors：heteroskedasticity-consistent (HC0) 估计  # .fit() → 用极大似然估计 (MLE) 拟合 Logit 模型。 # 并列另一种fit。
print(logit_result_robust.summary())

Optimization terminated successfully.
         Current function value: 0.363798
         Iterations 7
                            Logit Regression Results                           
Dep. Variable:     choice_is_starbucks   No. Observations:                10000
Model:                           Logit   Df Residuals:                     9997
Method:                            MLE   Df Model:                            2
Date:                 Sun, 21 Sep 2025   Pseudo R-squ.:                  0.4230
Time:                         01:51:36   Log-Likelihood:                -3638.0
converged:                        True   LL-Null:                       -6305.1
Covariance Type:                   HC0   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
price_diff                  -1.5569      0.112    -13.955      0.000    

### Predicted probabilities

In [29]:
df['presdicted_prob_starbucks'] = logit_result.predict(X)                      # 用回归结果 (logit_result) 对特征矩阵 X 进行预测。
print(df[['choice_is_starbucks', 'presdicted_prob_starbucks']].head(10))       # 显示前 10 行的实际选择 (choice_is_starbucks) 和预测概率 (presdicted_prob_starbucks)

   choice_is_starbucks  presdicted_prob_starbucks
0                    1                   0.721069
1                    0                   0.073876
2                    1                   0.135187
3                    0                   0.014750
4                    0                   0.050792
5                    0                   0.718698
6                    0                   0.185674
7                    0                   0.002713
8                    0                   0.001230
9                    1                   0.594925


### Classification Rate
what's classification rate?: the percentage of correct predictions made by the model.

In [41]:
df['predicted_choice'] = (df['presdicted_prob_starbucks'] >= 0.5).astype(int) 
# df['presdicted_prob_starbucks'] >= 0.5
# 这是一个布尔条件，意思是：
# 如果概率 ≥ 0.5 → True
# 如果概率 < 0.5 → False

# .astype(int)， 把布尔值转换成整数：
# True → 1
# False → 0

# Calculate classification rate: the percentage of correct predictions

misclassified = (df['predicted_choice'] != df['choice_is_starbucks']).mean()  # 计算预测错误的比例

print(misclassified)

0.1657


.mean 和 .mean() 的区别：

    .mean 这里表示 方法对象本身，而不是结果。你相当于把 函数 存在了 misclassified 里，而不是把“计算结果”存进去。

    .mean() 表示 调用方法，即执行计算

### Marginal Effects

In [ ]:
mfx = logit_result.get_margeff()  # 从 Logit 回归结果对象 (logit_result) 里，计算 边际效应 (marginal effects)。
print(mfx.summary())

        Logit Marginal Effects        
Dep. Variable:     choice_is_starbucks
Method:                           dydx
At:                            overall
                              dy/dx    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
price_diff                  -0.1799      0.012    -14.888      0.000      -0.204      -0.156
time_diff                   -0.0143      0.001    -28.522      0.000      -0.015      -0.013
price_income_interaction     0.0003      0.000      1.372      0.170      -0.000       0.001


### Market Share Simulation (Price + 1)

In [ ]:
df_sim = df.copy() # create a copy of the original dataframe for simulation  # 创建数据框的副本，用于模拟
df_sim['price_starbucks_new'] = df_sim['price_starbucks'] + 1 
df_sim['price_diff_new'] = df_sim['price_starbucks_new'] - df_sim['price_dunkin']

df_sim['price_income_interaction_new'] = df_sim['price_starbucks_new'] * df_sim['income'] - df_sim['price_dunkin'] * df_sim['income']

X_sim = df_sim[['price_diff_new', 'time_diff', 'price_income_interaction_new']] 
df_sim['predicted_prob_starbucks_new'] = logit_result.predict(X_sim)          # 用拟合好的模型对象对新特征矩阵做预测；        

old_market_share = df['choice_is_starbucks'].mean()    
new_market_share = df_sim['predicted_prob_starbucks_new'].mean()
print(f"Old Market Share of Starbucks: {old_market_share:.4f}")              # :.4f 表示保留 4 位小数。
print(f"New Market Share of Starbucks after $1 price increase: {new_market_share:.4f}")

Old Market Share of Starbucks: 0.3249
New Market Share of Starbucks after $1 price increase: 0.1781


### Market Share Simulation (Income + 10% increase)

In [32]:
# income increase
df_sim2 = df.copy()
df_sim2['income_new'] = df_sim2['income'] * 1.1
df_sim2['price_income_interaction_new'] = df_sim2['price_starbucks'] * df_sim2['income_new'] - df_sim2['price_dunkin'] * df_sim2['income_new']

# new X variables
X_sim2 = df_sim2[['price_diff', 'time_diff', 'price_income_interaction_new']]

# predicted probabilities with new income
df_sim2['predicted_prob_starbucks_new2'] = logit_result.predict(X_sim2)

# comare market shares
old_market_share2 = df['choice_is_starbucks'].mean()
new_market_share2 = df_sim2['predicted_prob_starbucks_new2'].mean()
print(f"Old Market Share of Starbucks: {old_market_share2:.4f}")
print(f"New Market Share of Starbucks after 10% income increase: {new_market_share2:.4f}")  

Old Market Share of Starbucks: 0.3249
New Market Share of Starbucks after 10% income increase: 0.3221


## Try Logit By Hand

In [33]:
from scipy.optimize import minimize

The @ operator 

In [45]:
# Define Two Matrices
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

print(f"\n{A}")
print(f"\n{B}")
print(f"\n{A @ B}")  # Matrix multiplication


[[1 2]
 [3 4]]

[[5 6]
 [7 8]]

[[19 22]
 [43 50]]


Define our log-likelihood function

In [ ]:
def log_likelihood(beta): #def ...: → 定义函数。
    """compute the log-likelihood for a binary logit model"""

    #compute mean utility for each observation
    V = X @ beta

    # logit choice Probabilities
    P = np.exp(V) / (1 + np.exp(V))      # np.exp(V) → 对数组逐元素取指数

    # avoid log(0)
    esp = 1e-12                          # esp = 1e-12 → 定义一个极小的常数。
    p = np.clip(P, esp, 1 - esp)         # np.clip(P, esp, 1 - esp) → 把 P 限制在 [esp, 1-esp] 范围内，避免出现 0 或 1

    # log-likelihood
    logL = y * np.log(p) + (1 - y) * np.log(1 - p)

    return -np.sum(logL)    # np.sum(logL) → 所有样本的 log-likelihood 总和。 # 前面加负号 -，是因为我们通常用最小化 (minimization) 方法来求解最大似然估计 (MLE)。 最大化 log-likelihood 等价于最小化它的负数。 （下面）

In [ ]:
# Initialize parameters
beta_init = np.zeros(X.shape[1])
# np.zeros(n) → 生成一个长度为 n 的全零数组。
# X.shape[1] → 返回 X 的列数（也就是变量个数）。 X.shape[0] → 行数，表示样本量（observation 个数）


# optimize the likelihood
result = minimize(log_likelihood, beta_init, method='BFGS')
# minimize → 来自 scipy.optimize，通用优化函数。
# 第一个参数：目标函数（这里是你写的 log_likelihood）。
# 第二个参数：初始值 beta_init。
# method='BFGS' → 指定优化算法是 BFGS（拟牛顿法）。

beta = result.x
# result 是一个 OptimizeResult 对象，类似字典。
# 常用属性：
# result.x → 最优解（这里就是估计的系数）。
# result.fun → 最优目标函数值。
# result.success → 是否收敛。
# result.message → 收敛信息。

print(beta)

[-1.55688815 -0.12355455  0.00237812]


## Try Probit By Hand

In [39]:
from scipy.stats import norm  # Import norm for Probit model

Define new likelihood function

In [ ]:
def log_likelihood(beta):
    """compute the log-likelihood for a binary probit model"""

    #compute mean utility for each observation
    V = X @ beta

    # Probit choice Probabilities
    P = norm.cdf(V)               # norm.cdf → 正态分布的累积分布函数 (CDF)。

    # avoid log(0)
    esp = 1e-12
    p = np.clip(P, esp, 1 - esp)

    # log-likelihood
    logL = y * np.log(p) + (1 - y) * np.log(1 - p)

    return -np.sum(logL)  # Negative for minimization   

In [41]:
# Initialize parameters
beta_init = np.zeros(X.shape[1])

# optimize the likelihood
result = minimize(log_likelihood, beta_init, method='BFGS')

beta = result.x

print(beta)

[-0.863595   -0.06963015  0.00106199]
